In [4]:
import tensorflow as tf
import numpy as np
import argparse
import pickle
import src.network as network
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score



for lr in [0.001, 0.0001]:
    for reg_alpha in [0.0001, 0.0]:
        for dropout_prob in [0.5, 1.0]:
            for embedding_size in [64, 128]:
                for batch_size in [32, 64]:
                    for hidden_cells in [64, 128]:
                        
                        epochs = 1000
                        use_pretrained_embeddings = embedding_size == 100

                        current_model_name = "model-{}-{}-{}-{}-{}-{}".format(
                          lr,
                          reg_alpha,
                          dropout_prob,
                          embedding_size,
                          batch_size,
                          hidden_cells
                        )
                        print(current_model_name)

                        # Load parameters
                        with open('./dumps/dump.pickle', 'rb') as handle:
                            dumped = pickle.load(handle)

                        vocab = dumped["vocab"]
                        X_train, Y_train = dumped["X_train"], dumped["Y_train"]
                        X_val, Y_val = dumped["X_val"], dumped["Y_val"]
                        assert(X_train.shape[0] == Y_train.shape[0])
                        assert(X_train.shape[1] == Y_train.shape[1])
                        assert(X_val.shape[0] == Y_val.shape[0])
                        assert(X_val.shape[1] == Y_val.shape[1])

                        # Load embeddings trained with Protovec
                        if use_pretrained_embeddings:
                            trained_embeddings = network.loadEmbeddings(vocab, "./train_embeddings/models/ngram-1.model", embedding_size)

                            assert(len(vocab.index_to_word) == len(vocab.word_to_index) == len(trained_embeddings))
                            assert(len(trained_embeddings[5]) == embedding_size)
                            #print("Embedding layer words: " + str(len(trained_embeddings)))
                        else:
                            trained_embeddings = None

                        # Reset TF graph
                        tf.reset_default_graph()

                        # Placeholders
                        tensor_X = tf.placeholder(tf.int32, (None, X_train.shape[1]), 'inputs')
                        tensor_Y = tf.placeholder(tf.int32, (None, None), 'outputs')
                        keep_prob = tf.placeholder(tf.float32, (None), 'dropout_keep')

                        # Create graph for the network
                        logits, mask, sequence_length = network.create_network(tensor_X, tensor_Y, keep_prob, vocab, embedding_size, hidden_cells, trained_embeddings)

                        # Cross entropy loss after softmax of logits with regularization
                        ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=tensor_Y) * mask
                        meaned = tf.reduce_mean(ce)
                        trainable_vars = tf.trainable_variables()
                        l2_reg = tf.reduce_sum([tf.nn.l2_loss(var) for var in trainable_vars])
                        loss = meaned + reg_alpha * l2_reg

                        # Using Adam optimizer for the update of the weights of the network with gradient clipping
                        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
                        gradients, variables = zip(*optimizer.compute_gradients(loss))
                        gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
                        optimize = optimizer.apply_gradients(zip(gradients, variables))

                        # Useful tensors
                        scores = tf.nn.softmax(logits)
                        predictions = tf.to_int32(tf.argmax(scores, axis=2))
                        accuracy = tf.contrib.metrics.accuracy(predictions, tensor_Y, weights=mask)

                        # Training data variables
                        iterations_training = max((len(X_train) // batch_size), 1)

                        # Validation data variables
                        max_val_acc = 0
                        consecutive_validation = 0

                        # Perform each epoch, shuffle training dataset
                        indices = list(range(len(X_train)))

                        with tf.Session() as sess:

                            # Initialize variables in the graph
                            saver = tf.train.Saver()
                            sess.run(tf.global_variables_initializer())
                            sess.run(tf.local_variables_initializer())

                            # Iterate over epochs
                            for i in range(epochs):

                                # Shuffle data (with random seed) to not train the network always with the same order
                                np.random.seed(42)
                                np.random.shuffle(indices)
                                X_train, Y_train = X_train[indices], Y_train[indices]

                                # Iterate over mini-batches
                                for j in range(0, len(X_train), batch_size):

                                    _ = sess.run(optimize, feed_dict={
                                                            tensor_X: X_train[j:j+batch_size],
                                                            tensor_Y: Y_train[j:j+batch_size],
                                                            keep_prob: dropout_prob })


                                    # Statistics on validation set
                                    if i >= 1 and ((j//batch_size) + 1) % ((iterations_training - 1)//2) == 0:
                                        avg_accuracy, avg_loss, pred = sess.run([accuracy, loss, predictions], feed_dict={ 
                                                                                                        tensor_X: X_val,
                                                                                                        tensor_Y: Y_val,
                                                                                                        keep_prob: 1.0 })

                                        # Save model if validation accuracy better
                                        if avg_accuracy > max_val_acc:
                                            print(consecutive_validation)
                                            consecutive_validation = 0
                                            max_val_acc = avg_accuracy
                                            print("VALIDATION loss: " + str(avg_loss) + ", accuracy: " + str(avg_accuracy))
                                            save_path = saver.save(sess, "./checkpoints/{}.ckpt".format(current_model_name))
                                        else:
                                            consecutive_validation += 1

                                if consecutive_validation >= 6:
                                    print("Early stopping")
                                    break


                        # Final statistics
                        with tf.Session() as sess:
                            saver = tf.train.Saver()
                            sess.run(tf.global_variables_initializer())
                            sess.run(tf.local_variables_initializer())
                            saver.restore(sess, "./checkpoints/{}.ckpt".format(current_model_name)) 

                            final_pred, final_true = np.array([]), np.array([])
                            for index_val in range(len(X_val)):
                                avg_accuracy, bb, pred = sess.run([accuracy, sequence_length, predictions], feed_dict={
                                                                                                                        tensor_X: [X_val[index_val]],
                                                                                                                        tensor_Y: [Y_val[index_val]],
                                                                                                                        keep_prob: 1.0  })

                                final_pred = np.append(final_pred, pred[0][:bb[0]])
                                final_true = np.append(final_true, Y_val[index_val][:bb[0]])

                            #print(Y_val[index_val][:bb[0]])
                            #print(pred[0][:bb[0]])

                        f = open("results.txt","a+")
                        acc = round(accuracy_score(final_true, final_pred), 3)
                        prec = round(precision_score(final_true, final_pred), 3)
                        rec = round(recall_score(final_true, final_pred), 3)
                        auc = round(roc_auc_score(final_true, final_pred), 3)
                        f.write(current_model_name + " " + str(acc) + " " + str(prec) + " " + str(rec) + " " + str(auc) + "\n")
                        f.close()

model-0.001-0.0001-0.5-64-32-64
0
VALIDATION loss: 0.27830145, accuracy: 0.453649
2
VALIDATION loss: 0.26366538, accuracy: 0.47943094
0
VALIDATION loss: 0.2448388, accuracy: 0.6148118
2
VALIDATION loss: 0.22915977, accuracy: 0.67094654
Early stopping
INFO:tensorflow:Restoring parameters from ./checkpoints/model-0.001-0.0001-0.5-64-32-64.ckpt
model-0.001-0.0001-0.5-64-32-128
0
VALIDATION loss: 0.27061602, accuracy: 0.52103126
3
VALIDATION loss: 0.25147256, accuracy: 0.5769606
2
VALIDATION loss: 0.23971717, accuracy: 0.6308356
Early stopping
INFO:tensorflow:Restoring parameters from ./checkpoints/model-0.001-0.0001-0.5-64-32-128.ckpt
model-0.001-0.0001-0.5-64-64-64
0
VALIDATION loss: 0.27870166, accuracy: 0.48482358
0
VALIDATION loss: 0.26440158, accuracy: 0.59914744
2
VALIDATION loss: 0.2505522, accuracy: 0.6017154
2
VALIDATION loss: 0.24306346, accuracy: 0.6311951
3
VALIDATION loss: 0.23188311, accuracy: 0.67936933
Early stopping
INFO:tensorflow:Restoring parameters from ./checkpoints/

0
VALIDATION loss: 0.20834574, accuracy: 0.72184277
0
VALIDATION loss: 0.2107782, accuracy: 0.72297263
2
VALIDATION loss: 0.21134247, accuracy: 0.7248729
1
VALIDATION loss: 0.20653224, accuracy: 0.73344976
Early stopping
INFO:tensorflow:Restoring parameters from ./checkpoints/model-0.001-0.0-0.5-64-32-128.ckpt
model-0.001-0.0-0.5-64-64-64
0
VALIDATION loss: 0.246218, accuracy: 0.43813878
0
VALIDATION loss: 0.23078294, accuracy: 0.5578039
2
VALIDATION loss: 0.21973233, accuracy: 0.58219916
4
VALIDATION loss: 0.21934745, accuracy: 0.6253402
0
VALIDATION loss: 0.20882215, accuracy: 0.6780854
1
VALIDATION loss: 0.2020676, accuracy: 0.7195316
Early stopping
INFO:tensorflow:Restoring parameters from ./checkpoints/model-0.001-0.0-0.5-64-64-64.ckpt
model-0.001-0.0-0.5-64-64-128
0
VALIDATION loss: 0.23881274, accuracy: 0.517693
1
VALIDATION loss: 0.22937705, accuracy: 0.53762007
1
VALIDATION loss: 0.2338701, accuracy: 0.54701865
1
VALIDATION loss: 0.22291623, accuracy: 0.6170202
0
VALIDATION lo

model-0.0001-0.0001-0.5-64-64-128
0
VALIDATION loss: 0.29952118, accuracy: 0.4824611
0
VALIDATION loss: 0.29550412, accuracy: 0.5266807
0
VALIDATION loss: 0.28830406, accuracy: 0.59657955
Early stopping
INFO:tensorflow:Restoring parameters from ./checkpoints/model-0.0001-0.0001-0.5-64-64-128.ckpt
model-0.0001-0.0001-0.5-128-32-64
0
VALIDATION loss: 0.3215084, accuracy: 0.40568024
0
VALIDATION loss: 0.31295833, accuracy: 0.47373018
0
VALIDATION loss: 0.3084848, accuracy: 0.5135843
5
VALIDATION loss: 0.3039033, accuracy: 0.52180165
Early stopping
INFO:tensorflow:Restoring parameters from ./checkpoints/model-0.0001-0.0001-0.5-128-32-64.ckpt
model-0.0001-0.0001-0.5-128-32-128
0
VALIDATION loss: 0.33380663, accuracy: 0.44471264
0
VALIDATION loss: 0.32236952, accuracy: 0.5428586
Early stopping
INFO:tensorflow:Restoring parameters from ./checkpoints/model-0.0001-0.0001-0.5-128-32-128.ckpt
model-0.0001-0.0001-0.5-128-64-64
0
VALIDATION loss: 0.30558175, accuracy: 0.57521445
1
VALIDATION loss: 